# Fetch from Openai function

In [67]:
import json
from openai import OpenAI
import toml
import streamlit as st

# Initialize the OpenAI client with the API key from the secrets file
api_key = st.secrets["api_key"]
client = OpenAI(api_key=api_key)

def fetch_random_cities():
    """
    Fetches details about two random cities using the OpenAI API.
    """
    prompt = """
    Provide a JSON object with the following details about two random capitals:
    - Name of the capital
    - Country of the capital
    - Distance between the two capitals in kilometers
    - Estimated flight time between the cities in hours
    - Fun fact about each cit
    - A detailed breakdown of walking and swimming time, assuming:
      - Humans walk 5 km per hour, walking 8 hours per day.
      - Swimming speed is 2 km per hour, swimming 6 hours per day.
      - The walking time is calculated based on an approximate percentage of the total distance being over land, e.g., 70% land and 30% sea. Adjust walking and swimming times accordingly.
    
    Example:
    {
        "capital_1": {"name": "Paris", "country": "France", "fun_fact": "Paris has a hidden vineyard in Montmartre."},
        "capital_2": {"name": "New York", "country": "USA", "fun_fact": "New York's Central Park is larger than Monaco."},
        "distance_km": 5837,
        "flight_time_hours": 7.2,
        "walking_swimming_time": "You will need 120 days walking + 60 days swimming."
    }
    """
    try:
        # Create a chat completion request
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a fact and geography expert and provide the requested data accurately and you dont favor popular capitals over others, the probability is even across all capitals."},
                {"role": "user", "content": prompt},
            ],
            temperature=1.0 #we make it 1.0 to be as random as possible
        )
        # Extract and parse the response content
        content = response.choices[0].message.content.strip()
        return json.loads(content)
    except json.JSONDecodeError:
        return "We are experiencing a server issue, play try again."
    except Exception as e:
        return f"An error occurred: {e}"

if __name__ == "__main__":
    cities_info = fetch_random_cities()
    print("City Details:", cities_info)


City Details: {'capital_1': {'name': 'Ulaanbaatar', 'country': 'Mongolia', 'fun_fact': 'Ulaanbaatar is the coldest capital city in the world with average temperatures below zero degrees celsius for more than half the year.'}, 'capital_2': {'name': 'Lisbon', 'country': 'Portugal', 'fun_fact': 'Lisbon is known for having the oldest bookstore in the world, Bertrand Bookstore, which opened in 1732.'}, 'distance_km': 7968, 'flight_time_hours': 10, 'walking_swimming_time': 'Land: 70%, Sea: 30%. You will need 140 days walking + 66 days swimming.'}


In [68]:
import json
from openai import OpenAI
import streamlit as st

# Initialize the OpenAI client with the API key from secrets
api_key = st.secrets["api_key"]
client = OpenAI(api_key=api_key)

def evaluate_guess(city_details, user_guess):
    """
    Evaluate the user's guess in the context of the city game.

    Args:
        city_details (dict): Dictionary containing city details and distance information.
        user_guess (str): The city guessed by the user.

    Returns:
        dict: Evaluation result including correctness, capital status, and distance.
    """
    # Construct the prompt for the OpenAI model
    prompt = f"""
    Reference City: {city_details['capital_1']['name']}
    Correct City: {city_details['capital_2']['name']}
    User Guess: {user_guess}

    Evaluate the user's guess following these instructions:
    -Is the guess correct?
    -Is it a capital of a country?
    -is the guessed city a real city (valid)? 
    -if the city does not exist, return null.  
    -if the city is a valid city (real) calculate the distance in kilometers from the reference city to the guessed city.
    -Provide the output in this JSON format:
    {{
        "guess_correct": <true/false>,
        "is_capital": <true/false>,
        "valid_city": <true/false>,
        "distance_to_reference_km": <distance or null>
    }}
    """

    try:
        # Use the chat completion API
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a geography and distance expert that evaluates accurately if a certain input is a city, capital, and how far is it from the given capital."},
                {"role": "user", "content": prompt},
            ],
            temperature=0  # Set temperature for deterministic responses
        )
        # Extract and parse the response content
        content = response.choices[0].message.content.strip()
        result = json.loads(content)  # Parse JSON-like string into a Python dictionary

        # Additional check: If the city is not valid, set the distance to None
        if not result.get("valid_city", False):
            result["distance_to_reference_km"] = None
        return result

    except json.JSONDecodeError:
        return {"error": "Failed to parse the response. Please try again."}
    except Exception as e:
        return {"error": str(e)}


In [70]:
city_details = {
    'capital_1': {'name': 'Santiago', 'country': 'Chile', 'fun_fact': 'Santiago is one of the most polluted cities in Latin America.'},
    'capital_2': {'name': 'Maputo', 'country': 'Mozambique', 'fun_fact': 'Maputo is known for its jacaranda-lined streets and vibrant arts scene.'},
    'distance_km': 6643,
    'flight_time_hours': 8.3,
    'walking_swimming_time': 'You will need 139 days walking + 68 days swimming.'
}
user_guess = "tunis"

result = evaluate_guess(city_details, user_guess)
print(result)


{'guess_correct': False, 'is_capital': True, 'valid_city': True, 'distance_to_reference_km': 11335}


# Evaluate guess quality function

# Working test code: 

In [5]:
from assets.utils import fetch_random_cities, guess_eval
import streamlit as st


def main():
    st.title("Guess the Distance Between Two Cities")

    if 'game_data' not in st.session_state:
        st.session_state.game_data = fetch_random_cities()
        st.session_state.guesses = []
        st.session_state.rounds = []

    game_data = st.session_state.game_data

    if isinstance(game_data, str):
        st.error(game_data)
        return

    city1 = game_data['city1']
    city2 = game_data['city2']
    target_distance = game_data['distance_km']

    st.write(f"Guess the distance between **{city1['name']}**, {city1['country']} and **{city2['name']}**, {city2['country']}.")

    with st.form(key='guess_form'):
        user_guess = st.number_input("Enter your guess in kilometers:", min_value=0)
        submit_button = st.form_submit_button(label='Submit')

    if submit_button:
        is_correct, difference = guess_eval(target_distance, user_guess)
        st.session_state.guesses.append(difference)

        if is_correct:
            st.success("Congratulations! Your guess is correct.")
            st.session_state.rounds.append({
                'target_distance': target_distance,
                'total_difference': sum(st.session_state.guesses),
                'num_guesses': len(st.session_state.guesses)
            })
            st.session_state.game_data = fetch_random_cities()
            st.session_state.guesses = []
        else:
            st.info(f"Your guess was off by {difference:.2f} km. Try again!")

if __name__ == "__main__":
    main()


2024-11-28 07:52:09.536 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-28 07:52:09.537 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-28 07:52:09.537 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-28 07:52:09.537 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-28 07:52:09.538 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-28 07:52:09.538 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-28 07:52:09.538 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-28 07:52:09.538 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar